In [82]:
import numpy as np
import matplotlib.pyplot as plt
import scipy 
import os
import pickle

In [47]:
import PyQt6.QtCore
os.environ["QT_API"] = "pyqt6"

In [139]:
raster_movie = scipy.io.loadmat('data/Caos_session1/rasters_movies.mat')['rasters']
raster_image = scipy.io.loadmat('data/Caos_session1/rasters_images.mat')['rasters_images']

movie_names = scipy.io.loadmat('data/Caos_session1/movie_names.mat')['imgs']
movie_names = np.asarray([movie_names[i][0][0] for i in range(len(movie_names))])
img_names = scipy.io.loadmat('data/Caos_session1/image_names.mat')['imgs_ims']
img_names = np.asarray([img_names[i][0][0] for i in range(len(img_names))])

spikeID_movie = np.squeeze(scipy.io.loadmat('data/Caos_session1/spikeID_movies.mat')['spike_movies'])
spikeID_ims = np.squeeze(scipy.io.loadmat('data/Caos_session1/spikeID_ims.mat')['spike_ims'])

In [140]:
spikeID_movie

array([ 1,  2,  3,  3,  3,  4,  4,  5,  6,  7,  7,  8,  8,  9,  9,  9,  9,
       10, 11, 12, 12, 12, 13, 14, 15, 16, 16, 17, 17, 18, 19, 20, 21, 22,
       23, 24, 25, 25, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 36,
       37, 38, 39, 40, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 51,
       52, 53, 54, 55, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 68, 69, 70, 71, 71, 71, 72, 73, 73, 74, 75, 76, 77, 78, 79, 79,
       80, 81, 81, 82, 83, 84, 85, 85, 86, 87, 88, 89, 90, 90, 91, 91, 92,
       92, 93, 93, 93, 94, 95, 96, 96], dtype=uint8)

In [141]:
def convert_channels(spikeID):

    channels =[]
    currunit = 1

    for i in range(len(spikeID)):
        if i ==0:
            currunit = 1
        else:
            if spikeID[i-1] == spikeID[i]:
                currunit+=1 
            else:
                currunit = 1

        channels.append(f"Channel{spikeID[i]}_Unit{currunit}")
    return channels

channels_movie = convert_channels(spikeID_movie)
channels_img = convert_channels(spikeID_ims)

## Analyze rasters and PSTH averaged over all trials

In [142]:
trial_avg_movie = np.mean(raster_movie, axis=2)
trial_avg_image = np.mean(raster_image, axis=2)

norm_trial_avg_movie = scipy.stats.zscore(trial_avg_movie, axis=1)
norm_trial_avg_image = scipy.stats.zscore(trial_avg_image, axis=1)

In [143]:
%matplotlib qt 

fig = plt.figure()

plt.imshow(norm_trial_avg_movie)
plt.xlabel('Normalized trial averaged activity')
plt.ylabel('Channel')
plt.colorbar()  # Add a colorbar
plt.title('Caos, session1, movies')
plt.show()

In [51]:
%matplotlib qt 

fig = plt.figure()

plt.imshow(norm_trial_avg_image)
plt.xlabel('Normalized trial averaged activity')
plt.ylabel('Channel')
plt.colorbar()  # Add a colorbar
plt.title('Caos, session1, images')
plt.show()

In [90]:
qual = input()

if int(qual) not in [1,2,3,4]:
    print('false')
else:
    print('true')

true


In [ ]:
%matplotlib inline

def check_PSTH(raster, channels, filename):
    qual_dict = {'Poor':[], 'Fair':[], 'Good':[], 'Excellent':[]}
    
    trial_avg_data = np.mean(raster, axis=2)
    sem = scipy.stats.sem(raster, axis=2)

    for i in range(trial_avg_data.shape[0]):
        fig = plt.figure()
        plt.plot(np.arange(trial_avg_data.shape[1]),trial_avg_data[i])
        plt.fill_between(np.arange(trial_avg_data.shape[1]),trial_avg_data[i]+1.96*sem[i], trial_avg_data[i]-1.96*sem[i], alpha=0.5)
        plt.show()

        qual = input("How good is this PSTH")

        while qual not in ['1','2','3','4']:
            qual = input('invalid input, should be one of 1, 2, 3, 4')
            
        if qual == '1':
            qual_dict['Poor'].append((i, channels[i]))

        elif qual == '2':
            qual_dict['Fair'].append((i, channels[i]))

        elif qual == '3':
            qual_dict['Good'].append((i, channels[i]))

        elif qual == '4': 
            qual_dict['Excellent'].append((i, channels[i]))

        plt.close()

    with open(f'{filename}.pickle', 'wb') as pick:
        pickle.dump(qual_dict, pick, protocol=pickle.HIGHEST_PROTOCOL)

    return qual_dict

movie_qualdict = check_PSTH(raster_movie, channels_movie, 'data/Caos_session1/movie_PSTH')

In [ ]:
image_qualdict = check_PSTH(raster_image, channels_img, 'data/Caos_session1/image_PSTH')

In [95]:
with open('data/Caos_session1/movie_PSTH.pickle', 'rb') as pick:
    movie_qualdict = pickle.load(pick)


In [ ]:
trial_avg_data = np.mean(raster_movie, axis=2)
sem = scipy.stats.sem(raster_movie, axis=2)

for value in movie_qualdict['Poor']:
    fig, plt.figure()

    i = value[0]

    plt.plot(np.arange(trial_avg_data.shape[1]),trial_avg_data[i])
    plt.fill_between(np.arange(trial_avg_data.shape[1]),trial_avg_data[i]+1.96*sem[i], trial_avg_data[i]-1.96*sem[i], alpha=0.5)

    #plt.plot(trial_avg_movie[value[0]])
    plt.title(f'{value[1]} , poor quality')

    plt.show()

In [149]:
print(len(movie_qualdict['Poor']))
print(len(movie_qualdict['Fair']))
print(len(movie_qualdict['Good']))
print(len(movie_qualdict['Excellent']))

print(len(image_qualdict['Poor']))
print(len(image_qualdict['Fair']))
print(len(image_qualdict['Good']))
print(len(image_qualdict['Excellent']))

23
18
31
55
29
14
17
66


In [ ]:
trial_avg_data = np.mean(raster_movie, axis=2)
sem = scipy.stats.sem(raster_movie, axis=2)

for value in movie_qualdict['Excellent']:
    fig, plt.figure()

    i = value[0]

    plt.plot(np.arange(trial_avg_data.shape[1]),trial_avg_data[i])
    plt.fill_between(np.arange(trial_avg_data.shape[1]),trial_avg_data[i]+1.96*sem[i], trial_avg_data[i]-1.96*sem[i], alpha=0.5)

    #plt.plot(trial_avg_movie[value[0]])
    plt.title(f'{value[1]} , Excellent quality')

    plt.show()

In [151]:
print(movie_qualdict['Excellent'])
print(image_qualdict['Excellent'])

[(8, 'Channel6_Unit1'), (10, 'Channel7_Unit2'), (22, 'Channel13_Unit1'), (24, 'Channel15_Unit1'), (26, 'Channel16_Unit2'), (41, 'Channel28_Unit1'), (46, 'Channel33_Unit1'), (47, 'Channel34_Unit1'), (48, 'Channel35_Unit1'), (49, 'Channel36_Unit1'), (51, 'Channel37_Unit1'), (52, 'Channel38_Unit1'), (53, 'Channel39_Unit1'), (54, 'Channel40_Unit1'), (56, 'Channel41_Unit1'), (57, 'Channel42_Unit1'), (58, 'Channel43_Unit1'), (59, 'Channel44_Unit1'), (60, 'Channel45_Unit1'), (61, 'Channel46_Unit1'), (62, 'Channel47_Unit1'), (63, 'Channel48_Unit1'), (64, 'Channel49_Unit1'), (65, 'Channel50_Unit1'), (66, 'Channel51_Unit1'), (67, 'Channel51_Unit2'), (68, 'Channel52_Unit1'), (69, 'Channel53_Unit1'), (70, 'Channel54_Unit1'), (72, 'Channel55_Unit2'), (73, 'Channel56_Unit1'), (74, 'Channel57_Unit1'), (76, 'Channel59_Unit1'), (77, 'Channel60_Unit1'), (78, 'Channel61_Unit1'), (79, 'Channel62_Unit1'), (80, 'Channel63_Unit1'), (81, 'Channel64_Unit1'), (82, 'Channel65_Unit1'), (83, 'Channel66_Unit1'), (8

In [152]:
movie_solid = [channel[1] for channel in movie_qualdict['Excellent']+movie_qualdict['Good']] 
movie_solid_ind = [channel[0] for channel in movie_qualdict['Excellent']+movie_qualdict['Good']] 
image_solid = [channel[1] for channel in image_qualdict['Excellent']+image_qualdict['Good']] 
image_solid_ind = [channel[0] for channel in image_qualdict['Excellent']+image_qualdict['Good']] 

In [153]:
mov_solid_chans = [movie_solid_ind[i] for i in range(len(movie_solid)) if movie_solid[i] in image_solid]
mov_solid_channames = [movie_solid[i] for i in range(len(movie_solid)) if movie_solid[i] in image_solid]
img_solid_chans = [image_solid_ind[i] for i in range(len(image_solid)) if image_solid[i] in movie_solid]
img_solid_channames = [image_solid[i] for i in range(len(image_solid)) if image_solid[i] in movie_solid]

In [154]:
mov_sol_chan_sort = sorted(zip(mov_solid_channames, mov_solid_chans), key=lambda x: x[0])
img_sol_chan_sort = sorted(zip(img_solid_channames, img_solid_chans), key=lambda x: x[0])

sort_mov_channames, sort_mov_indexes = zip(*mov_sol_chan_sort)
sort_img_channames, sort_img_indexes = zip(*img_sol_chan_sort)

In [158]:
print(len(sort_mov_channames))

74


In [159]:
print(sort_mov_channames)

('Channel10_Unit1', 'Channel11_Unit1', 'Channel12_Unit2', 'Channel15_Unit1', 'Channel16_Unit2', 'Channel18_Unit1', 'Channel1_Unit1', 'Channel21_Unit1', 'Channel23_Unit1', 'Channel24_Unit1', 'Channel28_Unit1', 'Channel30_Unit1', 'Channel33_Unit1', 'Channel34_Unit1', 'Channel35_Unit1', 'Channel36_Unit1', 'Channel37_Unit1', 'Channel38_Unit1', 'Channel40_Unit1', 'Channel41_Unit1', 'Channel42_Unit1', 'Channel43_Unit1', 'Channel44_Unit1', 'Channel45_Unit1', 'Channel46_Unit1', 'Channel47_Unit1', 'Channel48_Unit1', 'Channel49_Unit1', 'Channel50_Unit1', 'Channel51_Unit1', 'Channel51_Unit2', 'Channel52_Unit1', 'Channel53_Unit1', 'Channel54_Unit1', 'Channel55_Unit2', 'Channel56_Unit1', 'Channel57_Unit1', 'Channel58_Unit1', 'Channel59_Unit1', 'Channel60_Unit1', 'Channel61_Unit1', 'Channel62_Unit1', 'Channel63_Unit1', 'Channel64_Unit1', 'Channel65_Unit1', 'Channel66_Unit1', 'Channel67_Unit1', 'Channel68_Unit1', 'Channel68_Unit2', 'Channel69_Unit1', 'Channel6_Unit1', 'Channel71_Unit2', 'Channel72_Un

In [ ]:
trial_avg_data_mov = np.mean(raster_movie, axis=2)
sem_mov = scipy.stats.sem(raster_movie, axis=2)

trial_avg_data_img = np.mean(raster_image, axis=2)
sem_img = scipy.stats.sem(raster_image, axis=2)

for i in range(len(sort_mov_channames)):
    
    mov_ind = sort_mov_indexes[i]
    mov_name = sort_mov_channames[i]

    img_ind = sort_img_indexes[i]
    img_name = sort_img_channames[i]

    fig, plt.figure()

    plt.plot(np.arange(trial_avg_data_mov.shape[1]),trial_avg_data_mov[mov_ind])
    plt.fill_between(np.arange(trial_avg_data_mov.shape[1]),trial_avg_data_mov[mov_ind]+1.96*sem_mov[mov_ind], trial_avg_data_mov[mov_ind]-1.96*sem_mov[mov_ind], alpha=0.5)

    #plt.plot(trial_avg_movie[value[0]])
    plt.title(f'{mov_name} , Movie')

    plt.show()

    fig, plt.figure()

    plt.plot(np.arange(trial_avg_data_img.shape[1]),trial_avg_data_img[img_ind])
    plt.fill_between(np.arange(trial_avg_data_img.shape[1]),trial_avg_data_img[img_ind]+1.96*sem_img[img_ind], trial_avg_data_img[img_ind]-1.96*sem_img[img_ind], alpha=0.5)

    #plt.plot(trial_avg_movie[value[0]])
    plt.title(f'{img_name} , Image')

    plt.show()

In [162]:
mov_stim_indexes = {}

for i in range(len(movie_names)):

    mov = movie_names[i]
    if not mov in mov_stim_indexes.keys():
        mov_stim_indexes[mov] = [i]

    else:
        mov_stim_indexes[mov].append(i)

In [ ]:
def compare_movies_by_channel(raster_movie, movie, channelname, channelindex, mov_stim_indexes):

    fig, axs = plt.subplots(4,3)

    channel_data = np.squeeze(raster_movie[channelindex,:,:]) # time_bins x trials

    mov_opts = ['sped', 'first']
    mov_perts = ['original', 'inverted', 'noise']

    for i in len(mov_opts):
        for j in len(mov_perts):

            norm_name = f"{movie}_{mov_opts[i]}_{mov_perts[j]}"
            shuffle_name = f"{movie}_{mov_opts[i]}_{mov_perts[j]}_shuffle"

            norm_mov_inds = mov_stim_indexes[norm_name]
            shuffle_mov_inds = mov_stim_indexes[shuffle_name]

            norm_mov_trials = channel_data[:, norm_mov_inds]
            shuffle_mov_trials = channel_data[:, shuffle_mov_inds]

            norm_avg = np.mean(norm_mov_trials, axis=1)
            shuffle_avg = np.mean(shuffle_mov_trials, axis=1)

            norm_sem = scipy.stats.sem(norm_mov_trials, axis=1)
            shuffle_sem = scipy.stats.sem(shuffle_mov_trials, axis=1)

            axs[i*2,j].plot(norm_avg)
            axs[i*2,j].fill_between(np.arange(len(norm_avg), norm_avg+1.96*norm_sem, norm_avg-1.96*norm_sem))

            if i == 0:
                axs[i*2,j].set_title(f"{mov_opts[1]}_{mov_perts[j]}")
                axs[i*2+1,j].set_title(f"{mov_opts[1]}_{mov_perts[j]}_shuffle")
            else:
                axs[i*2,j].set_title(f"{mov_opts[0]}_{mov_perts[j]}")
                axs[i*2+1,j].set_title(f"{mov_opts[0]}_{mov_perts[j]}_shuffle")

            axs[i*2+1,j].plot(shuffle_avg)
            axs[i*2+1,j].fill_between(np.arange(len(shuffle_avg), shuffle_avg+1.96*shuffle_sem, shuffle_avg-1.96*shuffle_sem))

    fig.suptitle(f"PSTHs for {movie}, {channelname}")

    


In [163]:
mov_stim_indexes

{np.str_('ambulance_sped_inverted_shuffle'): [0, 34, 41, 109, 133, 150, 207],
 np.str_('komodo_first_original_shuffle'): [1, 46, 86, 124, 147, 182, 201],
 np.str_('komodo_first_noise_shuffle'): [2, 51, 106, 132, 154, 195],
 np.str_('komodo_first_original'): [3, 38, 95, 120, 158, 198],
 np.str_('ambulance_sped_noise_shuffle'): [4, 50, 77, 114, 162, 220, 224],
 np.str_('komodo_first_inverted'): [5, 57, 73, 74, 126, 156, 208],
 np.str_('ambulance_sped_original_shuffle'): [6, 65, 101, 121, 153, 185],
 np.str_('komodo_sped_noise_shuffle'): [7, 61, 83, 115, 163, 213],
 np.str_('ambulance_sped_noise'): [8, 66, 93, 122, 172, 191],
 np.str_('ambulance_sped_inverted'): [9, 49, 85, 127, 166, 192, 221],
 np.str_('komodo_sped_inverted_shuffle'): [10, 56, 97, 113, 164, 200],
 np.str_('macaque_eating_sped_inverted'): [11, 54, 94, 143, 183, 187],
 np.str_('macaque_eating_first_inverted'): [12, 52, 100, 138, 175, 216],
 np.str_('ambulance_first_noise_shuffle'): [13, 69, 82, 112, 159, 189],
 np.str_('ko